# ACS Analysis

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/Users/jamesgeiger/University of Nebraska Omaha/FALL 2021/ECON 4320/project/nhgis0005_csv/nhgis0005_ds244_20195_blck_grp_E.csv')

In [3]:
# rename columns according to IPUMS codebook
data = data.rename(columns={
    # Race
    'ALUCE001':    'Total',
    'ALUCE002':    'White alone',
    'ALUCE003':    'Black or African American alone',
    'ALUCE004':    'American Indian and Alaska Native alone',
    'ALUCE005':    'Asian alone',
    'ALUCE006':    'Native Hawaiian and Other Pacific Islander alone',
    'ALUCE007':    'Some other race alone',
    'ALUCE008':    'Two or more races',
    'ALUCE009':    'Two or more races: Two races including Some other race',
    'ALUCE010':    'Two or more races: Two races excluding Some other race, and three or more races',
    # Poverty
    'ALWVE001':    'Poverty total',
    'ALWVE002':    'Under .50',
    'ALWVE003':    '.50 to .99',
    'ALWVE004':    '1.00 to 1.24',
    'ALWVE005':    '1.25 to 1.49',
    'ALWVE006':    '1.50 to 1.84',
    'ALWVE007':    '1.85 to 1.99',
    'ALWVE008':    '2.00 and over',
    # Income
    'ALW1E001':    'Median household income',
    # Vehicles
    'AL0NE001':    'Vehicles total',
    'AL0NE002':    'Owner occupied',
    'AL0NE003':    'Owner occupied: No vehicle available',
    'AL0NE004':    'Owner occupied: 1 vehicle available',
    'AL0NE005':    'Owner occupied: 2 vehicles available',
    'AL0NE006':    'Owner occupied: 3 vehicles available',
    'AL0NE007':    'Owner occupied: 4 vehicles available',
    'AL0NE008':    'Owner occupied: 5 or more vehicles available',
    'AL0NE009':    'Renter occupied',
    'AL0NE010':    'Renter occupied: No vehicle available',
    'AL0NE011':    'Renter occupied: 1 vehicle available',
    'AL0NE012':    'Renter occupied: 2 vehicles available',
    'AL0NE013':    'Renter occupied: 3 vehicles available',
    'AL0NE014':    'Renter occupied: 4 vehicles available',
    'AL0NE015':    'Renter occupied: 5 or more vehicles available'
})

In [4]:
data['Percent Population NonWhite'] = ( data['Total'] - data['White alone'] ) / data['Total']

In [5]:
data['Percent Popoulation Impoverished/At Risk'] = ( data['Under .50'] + data['.50 to .99'] + data['1.00 to 1.24'] + data['1.25 to 1.49'] ) / data['Poverty total']

In [6]:
data['Percent Population Without Transportation'] = ( data['Owner occupied: No vehicle available'] + data['Renter occupied: No vehicle available'] ) / data['Vehicles total']

In [41]:
data['Income as percent of maximum'] = data['Median household income']/max(data['Median household income'])
data['CensusTractID'] = data['GEOID'].str[7:18].astype(int)

In [42]:
# incorporate USDA food desert definitions
food_access = pd.read_csv('/Users/jamesgeiger/University of Nebraska Omaha/FALL 2021/ECON 4320/project/food_access_data.csv')

In [43]:
food_access = food_access[(food_access['State'] == 'Nebraska')]
food_access = food_access[(food_access['County'] == 'Douglas County') | (food_access['County'] == 'Sarpy County')]
food_access = food_access[['CensusTract','LILATracts_1And10', 'LILATracts_halfAnd10', 'LILATracts_1And20', 'LILATracts_Vehicle']]

In [44]:
data = data.merge(food_access, left_on='CensusTractID', right_on='CensusTract', how='left')

In [45]:
data.to_csv('/Users/jamesgeiger/University of Nebraska Omaha/FALL 2021/ECON 4320/project/acs_data.csv')

# Places Analysis

In [9]:
import geopandas as gpd
from shapely.ops import nearest_points

In [10]:
places_df = pd.read_csv('/Users/jamesgeiger/University of Nebraska Omaha/FALL 2021/ECON 4320/project/places.csv')

In [11]:
places = gpd.GeoDataFrame(places_df, geometry=gpd.points_from_xy(places_df.latitude, places_df.longitude))

In [12]:
blocks_gdf = gpd.read_file('/Users/jamesgeiger/University of Nebraska Omaha/FALL 2021/ECON 4320/project/nhgis0004_shape/nhgis0004_shapefile_tl2019_310_blck_grp_2019/NE_blck_grp_2019.shp', crs="EPSG:4236")

In [13]:
blocks = gpd.GeoDataFrame(blocks_gdf, geometry=gpd.points_from_xy(blocks_gdf.INTPTLAT, blocks_gdf.INTPTLON))

In [14]:
def nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
    """Find the nearest point and return the corresponding value from specified column."""
    # Find the geometry that is closest
    nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
    # Get the corresponding value from df2 (matching is based on the geometry)
    value = df2[nearest][src_column].values[0]
    return value

In [15]:
unary_union = places.unary_union

In [16]:
blocks['nearest_placekey'] = blocks.apply(nearest, geom_union=unary_union, df1=blocks, df2=places, src_column='placekey', axis=1)
blocks['nearest_place_point'] = blocks.apply(nearest, geom_union=unary_union, df1=blocks, df2=places, src_column='geometry', axis=1)

In [17]:
blocks = blocks.merge(data, on="GISJOIN")
blocks = blocks[(blocks['COUNTYFP'] == '055')|(blocks['COUNTYFP']=='153')]

In [18]:
places = places.merge(blocks, left_on="placekey", right_on="nearest_placekey")

In [19]:
plcs = places.groupby(['placekey']).sum().reset_index()

In [20]:
plcs = plcs[['placekey','Total']].rename(columns={'Total':'Service Catchment Total'})

In [21]:
places_df.merge(plcs).to_csv('places_2.csv')